# Introduction

QA plots for the Silicon Seeding

In [1]:
# imports to write dynamic markdown contents
import os
from IPython.display import display, Markdown, Latex
from IPython.display import HTML

In [2]:
# turn off/on code for the result HTML page

display(Markdown('*For the result HTML page:* '))
    
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 $('div.jp-CodeMirrorEditor').hide(); // for newer versions of nbviewer.org
 } else {
 $('div.input').show();
 $('div.jp-CodeMirrorEditor').show(); // for refer versions of nbviewer.org
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

*For the result HTML page:* 

In [3]:
import os.path

# readme file of the macros, available if run under JenkinsCI
# https://github.com/sPHENIX-Collaboration/utilities/blob/master/jenkins/built-test/test-tracking-qa.sh 

macro_markdown = 'Fun4All-macros-README.md'

if os.path.isfile(macro_markdown) :
    with open(macro_markdown, 'r') as file:
        display(Markdown(file.read()))

## `pyROOT` env check

In [4]:
import ROOT

OFFLINE_MAIN = os.getenv("OFFLINE_MAIN")
if OFFLINE_MAIN is not None:
    display(Markdown(f"via sPHENIX software distribution at `{OFFLINE_MAIN}`"))

via sPHENIX software distribution at `/cvmfs/sphenix.sdcc.bnl.gov/alma9.2-gcc-14.2.0/release/release_new/new.11`

## Plotting source code

In [5]:
import subprocess

try:
    git_url = \
        subprocess.run(['git','remote','get-url','origin'], stdout=subprocess.PIPE)\
        .stdout.decode('utf-8').strip()\
        .replace('git@github.com:','https://github.com/')

    display(Markdown(f"View the source code repository at {git_url}"))
except: # catch *all* exceptions
    #     well do nothing
    pass

View the source code repository at https://github.com/sPHENIX-Collaboration/QA-gallery.git

## JenkinsCI information (if available)

In [6]:
display(Markdown('Some further details about the QA run, if executed under the Jenkins CI:'))

checkrun_repo_commit = os.getenv("checkrun_repo_commit")
if checkrun_repo_commit is not None:
    display(Markdown(f"* The commit being checked is {checkrun_repo_commit}"))
        
ghprbPullLink = os.getenv("ghprbPullLink")
if ghprbPullLink is not None:
    display(Markdown(f"* Link to the pull request: {ghprbPullLink}"))

BUILD_URL =  os.getenv("BUILD_URL")
if BUILD_URL is not None:
    display(Markdown(f"* Link to the build: {BUILD_URL}"))

git_url_macros =  os.getenv("git_url_macros")
sha_macros =  os.getenv("sha_macros")
if git_url_macros is not None:
    display(Markdown(f"* Git repo for macros: {git_url_macros} , which merges `{sha_macros}` and the QA tracking branch"))

RUN_ARTIFACTS_DISPLAY_URL = os.getenv("RUN_ARTIFACTS_DISPLAY_URL")
if RUN_ARTIFACTS_DISPLAY_URL is not None:
    display(Markdown(f"* Download the QA ROOT files: {RUN_ARTIFACTS_DISPLAY_URL}"))

JENKINS_URL  = os.getenv("JENKINS_URL")
if JENKINS_URL is not None:
    display(Markdown(f"Automatically generated by [sPHENIX Jenkins continuous integration]({JENKINS_URL}) [![sPHENIX](https://raw.githubusercontent.com/sPHENIX-Collaboration/utilities/master/jenkins/material/sphenix-logo-white-bg-72p.png)](https://www.sphenix.bnl.gov/web/) &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; [![jenkins.io](https://raw.githubusercontent.com/sPHENIX-Collaboration/utilities/master/jenkins/material/jenkins_logo_title-72p.png)](https://jenkins.io/)"))

Some further details about the QA run, if executed under the Jenkins CI:

* The commit being checked is sPHENIX-Collaboration/macros/4c48e180b40cf6bdb74b62417454a66f4e9ca383

* Link to the build: https://web.sdcc.bnl.gov/jenkins-sphenix/job/sPHENIX/job/test-tracking-reconstruction-prdf/929/

* Git repo for macros: https://github.com/sPHENIX-Collaboration/macros.git , which merges `origin/pr/1200/merge` and the QA tracking branch

* Download the QA ROOT files: https://web.sdcc.bnl.gov/jenkins-sphenix/job/sPHENIX/job/test-tracking-reconstruction-prdf/929/display/redirect?page=artifacts

Automatically generated by [sPHENIX Jenkins continuous integration](https://web.sdcc.bnl.gov/jenkins-sphenix/) [![sPHENIX](https://raw.githubusercontent.com/sPHENIX-Collaboration/utilities/master/jenkins/material/sphenix-logo-white-bg-72p.png)](https://www.sphenix.bnl.gov/web/) &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; [![jenkins.io](https://raw.githubusercontent.com/sPHENIX-Collaboration/utilities/master/jenkins/material/jenkins_logo_title-72p.png)](https://jenkins.io/)


# Initialization

In [7]:
%%cpp -d

#include "QA_Draw_Utility.C"

#include <sPhenixStyle.C>

#include <TFile.h>
#include <TLine.h>
#include <TString.h>
#include <TTree.h>
#include <cassert>
#include <cmath>

In [8]:
%%cpp

SetsPhenixStyle();
TVirtualFitter::SetDefaultFitter("Minuit2");

// test sPHENIX lib load
// gSystem->Load("libg4eval.so");

// test libs
// gSystem->ListLibraries();

sPhenixStyle: Applying nominal settings.
sPhenixStyle: ROOT6 mode


In [9]:
%jsroot on

## Inputs and file checks

In [10]:
qa_file_name_new = os.getenv("qa_file_name_new")
if qa_file_name_new is None:
    qa_file_name_new = "prdf_reconstruction53877_qa.root"
    display(Markdown(f"`qa_file_name_new` env not set. use the default `qa_file_name_new={qa_file_name_new}`"))

qa_file_name_ref = os.getenv("qa_file_name_ref")
if qa_file_name_ref is None:
    qa_file_name_ref = "prdf_reconstruction53877_qa.root"
    display(Markdown(f"`qa_file_name_ref` env not set. use the default `qa_file_name_ref={qa_file_name_ref}`"))
elif qa_file_name_ref == 'None':
    qa_file_name_ref = None
    qa_file_name_ref = "prdf_reconstruction53877_qa.root"
    display(Markdown(f"`qa_file_name_ref` = None and we are set to not to use the reference histograms"))
    

display(Markdown(f"Opening QA file at `{qa_file_name_new}`"))
ROOT.gInterpreter.ProcessLine(f"TFile *qa_file_new = new TFile(\"{qa_file_name_new}\");")
ROOT.gInterpreter.ProcessLine(f"const char * qa_file_name_new = \"{qa_file_name_new}\";")

if qa_file_name_ref is not None:
#     qa_file_ref = ROOT.TFile.Open(qa_file_name_ref);

#     assert qa_file_ref.IsOpen()
    display(Markdown(f"Opening QA reference file at `{qa_file_name_ref}`"))
    ROOT.gInterpreter.ProcessLine(f"TFile *qa_file_ref = new TFile(\"{qa_file_name_ref}\");")
    ROOT.gInterpreter.ProcessLine(f"const char *  qa_file_name_ref = \"{qa_file_name_ref}\";")
else:
    ROOT.gInterpreter.ProcessLine(f"TFile *qa_file_ref = nullptr;")
    ROOT.gInterpreter.ProcessLine(f"const char *  qa_file_name_ref = nullptr;")

Opening QA file at `/var/lib/jenkins/workspace/sPHENIX/test-tracking-reconstruction-prdf/macros/TrackingProduction/prdf_reconstruction53877_qa.root`

Opening QA reference file at `reference/prdf_reconstruction53877_qa.root`

In [11]:
%%cpp




    TCanvas* m_canvas_chip = new TCanvas("m_canvas_chip", "INTT Plots 0", 1000, 800);
    m_canvas_chip->Divide(2, 2);
  
   TH1F *nrecoseedsneg_new = (TH1F*)qa_file_new->Get("h_TpcSeedsQA_nrecotracks1d_neg");
   TH1F *nrecoseedspos_new = (TH1F*)qa_file_new->Get("h_TpcSeedsQA_nrecotracks1d_pos");
   TH1F *ntpcpos_new = (TH1F*)qa_file_new->Get("h_TpcSeedsQA_ntpc_fullpt_pos");
   TH1F *ntpcneg_new = (TH1F*)qa_file_new->Get("h_TpcSeedsQA_ntpc_fullpt_neg");
   TH1F *ntpotpos_new = (TH1F*)qa_file_new->Get("h_TpcSeedsQA_ntpot_pos");
   TH1F *ntpotneg_new = (TH1F*)qa_file_new->Get("h_TpcSeedsQA_ntpot_neg");
   TH1F *pt_neg_new = (TH1F*)qa_file_new->Get("h_TpcSeedsQA_pt_neg");
   TH1F *pt_pos_new = (TH1F*)qa_file_new->Get("h_TpcSeedsQA_pt_pos");
   TH1F *vx_new = (TH1F*)qa_file_new->Get("h_TpcSeedsQA_vx");
   TH1F *vy_new = (TH1F*)qa_file_new->Get("h_TpcSeedsQA_vy");
   TH1F *vz_new = (TH1F*)qa_file_new->Get("h_TpcSeedsQA_vz");
   TH1F *dedx_new = (TH1F*)qa_file_new->Get("h_TpcSeedsQA_mip_dedx");
   TH1F *opf_neg_new = (TH1F*)qa_file_new->Get("h_TpcSeedsQA_cluster_phisize1_fraction_neg");
   TH1F *opf_pos_new = (TH1F*)qa_file_new->Get("h_TpcSeedsQA_cluster_phisize1_fraction_pos");
   
   TH1F *nrecoseedsneg_ref = (TH1F*)qa_file_ref->Get("h_TpcSeedsQA_nrecotracks1d_neg");
   TH1F *nrecoseedspos_ref = (TH1F*)qa_file_ref->Get("h_TpcSeedsQA_nrecotracks1d_pos");
   TH1F *ntpcpos_ref = (TH1F*)qa_file_ref->Get("h_TpcSeedsQA_ntpc_fullpt_pos");
   TH1F *ntpcneg_ref = (TH1F*)qa_file_ref->Get("h_TpcSeedsQA_ntpc_fullpt_neg");
   TH1F *ntpotpos_ref = (TH1F*)qa_file_ref->Get("h_TpcSeedsQA_ntpot_pos");
   TH1F *ntpotneg_ref = (TH1F*)qa_file_ref->Get("h_TpcSeedsQA_ntpot_neg");
   TH1F *pt_neg_ref = (TH1F*)qa_file_ref->Get("h_TpcSeedsQA_pt_neg");
   TH1F *pt_pos_ref = (TH1F*)qa_file_ref->Get("h_TpcSeedsQA_pt_pos");
   TH1F *vx_ref = (TH1F*)qa_file_ref->Get("h_TpcSeedsQA_vx");
   TH1F *vy_ref = (TH1F*)qa_file_ref->Get("h_TpcSeedsQA_vy");
   TH1F *vz_ref = (TH1F*)qa_file_ref->Get("h_TpcSeedsQA_vz");
   TH1F *dedx_ref = (TH1F*)qa_file_ref->Get("h_TpcSeedsQA_mip_dedx");
   TH1F *opf_neg_ref = (TH1F*)qa_file_ref->Get("h_TpcSeedsQA_cluster_phisize1_fraction_neg");
   TH1F *opf_pos_ref = (TH1F*)qa_file_ref->Get("h_TpcSeedsQA_cluster_phisize1_fraction_pos");
   
   m_canvas_chip->cd(1);
   DrawReference(nrecoseedsneg_new, nrecoseedsneg_ref);
    m_canvas_chip->cd(2);
    DrawReference(nrecoseedspos_new, nrecoseedspos_ref);
    m_canvas_chip->cd(3);
    DrawReference(pt_neg_new, pt_neg_ref);
    m_canvas_chip->cd(4);
    DrawReference(pt_pos_new, pt_pos_ref);
    
   
  
    m_canvas_chip->Update();
 
    std::cout << "DrawChipInfo completed" << std::endl;


DrawChipInfo completed


#### TPC seed nTPC and nTPOT

In [12]:
%%cpp

TCanvas *can2 = new TCanvas("can2","can2",1000,800);

can2->Divide(2,2);
can2->cd(1);
DrawReference(ntpcneg_new, ntpcneg_ref);
can2->cd(2);
DrawReference(ntpcpos_new, ntpcpos_ref);
can2->cd(3);
DrawReference(ntpotneg_new, ntpotneg_ref);
can2->cd(4);
DrawReference(ntpotpos_new, ntpotpos_ref);

can2->Update();

#### TPC Seed Vertex Positions

In [13]:
%%cpp

TCanvas *can3 = new TCanvas("can3","can3",1000,800);

can3->Divide(3,1);

can3->cd(1);
vx_new->GetXaxis()->SetRangeUser(-1,1);
DrawReference(vx_new, vx_ref);
can3->cd(2);
vy_new->GetXaxis()->SetRangeUser(-1,1);
DrawReference(vy_new, vy_ref);
can3->cd(3);
DrawReference(vz_new, vz_ref);

can3->Update();

#### TPC OPFs and dE/dx

In [14]:
%%cpp

TCanvas *can4 = new TCanvas("can4","can4",1000,800);
can4->Divide(3,1);


can4->cd(1);
DrawReference(dedx_new, dedx_ref);
can4->cd(2);
DrawReference(opf_neg_new, opf_neg_ref);
can4->cd(3);
DrawReference(opf_pos_new, opf_pos_ref);

can4->Update();

In [15]:
%%cpp

KSTestSummary::getInstance()->make_summary_txt("QA-tpcseeds.txt");
KSTestSummary::getInstance()->make_summary_TCanvas()->Draw();

This notebook contains 14 KSTets: combined Chi2/nDoF = 2.93605 / 28, and combined __p-Value = 1__
